<a href="https://colab.research.google.com/github/gtimokhina/Machine-Learning/blob/master/ContentStatistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Factor: Content Statistics**

In [0]:
import spacy
import pickle
import pandas as pd

In [0]:
from google.colab import files
#uploaded = files.upload()


In [0]:

colnames = ['jsonid', 'label', 'headline_text', 'subject', 'speaker', 'speakerjobtitle', 'stateinfo','partyaffiliation', 'barelytruecounts', 'falsecounts','halftruecounts','mostlytrueocunts','pantsonfirecounts','context']

df = pd.read_csv('train.tsv', sep='\t', names = colnames, error_bad_lines=False)

In [0]:
nlp = spacy.load('en_core_web_sm')


In [0]:
# first statistics: length of headline text
# new column length
df['length'] = df['headline_text'].apply(lambda s:len( s ) - s.count(" "))


In [0]:
# second statistics: number of sentences in the article
def getCountOfSentences(text):
    document = nlp(text)
    sentences_in_article = [sen for sen in doc.sents]
    return len(sentences_in_article)
df['count_sentences'] = df['headline_text'].apply(lambda s:len( s ) - s.count(" "))

In [16]:
# third statistics: Lexical diversity, a measure of the richness of the authorss vocabulary
def lexicalDeversity(text):
  import nltk
  nltk.download('punkt')
  words = nltk.word_tokenize(text)
  ld = len(words) / float(len(set(words)))
  return round(ld,2)

# we will have new column 
df['lex_diversity'] = df['headline_text'].apply(lexicalDeversity)


Streaming output truncated to the last 5000 lines.
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data

In [0]:
# fourth statistics: Entity detection, also named as entity recognition, 
# identifies important elements like places, people, organizations, and languages within an input string of text. 
from spacy import displacy
def entity_count(text):
  document = nlp(text)
  entities=[(k, k.label_, k.label) for k in document.ents]
  return len(entities)

df['entity_count'] = df['headline_text'].apply(entity_count)


In [0]:
# Still need some work: fifth statistics: number of POS(part of Speech)
def getCountOfPOS(text):
    document = nlp(text)
    pos = document.count_by(spacy.attrs.POS)
    return pos

# we will have new column 
df['POS_Info']=df.headline_text.apply(getCountOfPOS)
dict_POS = df.POS_Info

list_keys =[]
for item in dict_POS:
    for key,value in item.items():
        list_keys.append(key)

list_keys = list(set(list_keys))



In [0]:
list_of_zeros = [0]*len(df)

In [0]:
df_pos_default_count = pd.DataFrame(
    list(zip(list_of_zeros, list_of_zeros,list_of_zeros,list_of_zeros,list_of_zeros,
             list_of_zeros,list_of_zeros,list_of_zeros,list_of_zeros,list_of_zeros,
             list_of_zeros,list_of_zeros,list_of_zeros,list_of_zeros,list_of_zeros,
             list_of_zeros, list_of_zeros, list_of_zeros)), columns =list_keys)

In [0]:
row =0
for item in dict_POS:
    for key,value in item.items():
        df_pos_default_count.loc[row,key]=value
    row = row +1

In [22]:
df_pos_default_count.head()


,96,97,98,99,100,101,103,84,85,86,87,89,90,91,92,93,94,95
0,2,2,0,0,2,0,0,2,1,0,0,0,1,0,4,0,0,0
1,4,4,0,0,4,0,0,1,3,2,1,0,2,0,5,0,1,1
2,7,3,0,0,3,0,0,0,4,0,0,0,2,0,2,0,1,0
3,0,1,0,0,1,0,0,2,0,0,1,0,0,0,7,0,1,0
4,0,1,0,0,1,0,0,1,2,0,0,0,3,0,3,0,0,0


In [24]:


df_stat = pd.merge(df,df_pos_default_count,left_index =True ,right_index=True)
df_stat.head()

,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context,length,count_sentences,lex_diversity,entity_count,POS_Info,96,97,98,99,100,101,103,84,85,86,87,89,90,91,92,93,94,95
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,72,72,1.00,1,"{100: 2, 90: 1, 96: 2, 84: 2, 92: 4, 97: 2, 85...",2,2,0,0,2,0,0,2,1,0,0,0,1,0,4,0,0,0
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,118,118,1.04,1,"{86: 2, 87: 1, 90: 2, 92: 5, 85: 3, 97: 4, 95:...",4,4,0,0,4,0,0,1,3,2,1,0,2,0,5,0,1,1
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,87,87,1.05,4,"{96: 7, 100: 3, 85: 4, 97: 3, 94: 1, 90: 2, 92...",7,3,0,0,3,0,0,0,4,0,0,0,2,0,2,0,1,0
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,67,67,1.00,0,"{92: 7, 87: 1, 84: 2, 94: 1, 100: 1, 97: 1}",0,1,0,0,1,0,0,2,0,0,1,0,0,0,7,0,1,0
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,45,45,1.00,0,"{90: 3, 84: 1, 92: 3, 100: 1, 85: 2, 97: 1}",0,1,0,0,1,0,0,1,2,0,0,0,3,0,3,0,0,0
